<a href="https://colab.research.google.com/github/Nov05/DS-Unit-2-Sprint-4-Project/blob/master/notebooks/02_plotly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# created by nov05 on 2019-08-01
# Nov05/DS-Unit-2-Sprint-4-Project/
# notebooks/

plotly example  https://colab.research.google.com/drive/1oijny-_J2K-7ma6jNq9jAp7C_tzNB27W  
https://medium.com/@plotlygraphs/plotly-py-4-0-is-here-offline-only-express-first-displayable-anywhere-fc444e5659ee  
https://community.plot.ly/t/introducing-plotly-py-4-0-0/25639  

https://plot.ly/python/figure-factory-subplots/

In [1]:
# !pip uninstall plotly
# !pip install plotly==3.6.1
!pip install plotly==4.0.0

     |████████████████████████████████| 6.8MB 2.8MB/s 
  Found existing installation: plotly 3.6.1
    Uninstalling plotly-3.6.1:
      Successfully uninstalled plotly-3.6.1


In [0]:
import plotly

# import plotly.plotly as py
## ImportError: 
## The plotly.plotly module is deprecated,
## please install the chart-studio package and use the
## chart_studio.plotly module instead. 

# import plotly.graph_objs as go # plotly 3.6.1
import plotly.graph_objects as go # plotly4.0.0
# plot = online mode, iplot = offline mode
from plotly.offline import init_notebook_mode, plot, iplot
from plotly.graph_objs import Histogram2dContour, Scatter
from plotly.graph_objs.histogram2dcontour import Contours
from plotly.graph_objs.scatter import Marker

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.figure_factory as ff

import plotly.tools as tls
from plotly.subplots import make_subplots

from joblib import dump, load

In [3]:
import plotly
print("plotly", plotly.__version__)
# make sure it is version 4.0.0

plotly 4.0.0


In [0]:
# for plotly 4.0.0 you won't need this code block
# define this function for plotly and plotly express displaying
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
//// both would work, enjoy plotly and plotly express in colab
//               plotly: 'https://cdn.plot.ly/plotly-1.5.1.min.js?noext',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))
# init_notebook_mode(connected=True)

# then call the function configure_plotly_browser_state() 
# in EVERY code block where there is 
# a Plotly plot (each will be an iframe) 

# import and merge datasets

In [0]:
url_tasks = "https://raw.githubusercontent.com/Derek-Jones/SiP_dataset/master/Sip-task-info.csv"
url_dates = "https://raw.githubusercontent.com/Derek-Jones/SiP_dataset/master/est-act-dates.csv"
tasks  = pd.read_csv(url_tasks, encoding='iso-8859-1')
dates  = pd.read_csv(url_dates, encoding='iso-8859-1')

# merge two datasets
# there are duplicate TaskNumbers in both datasets
# there are duplicate rows in "dates" dataset
alldata = tasks.merge(right=dates[-dates.duplicated()], on='TaskNumber', how='inner')

# Project Breakdown Number
breakdown = (alldata[['ProjectCode', 'ProjectBreakdownCode']]
             .groupby('ProjectCode').nunique()
             .drop('ProjectCode', axis=1)
             .reset_index())
# Developer Performance
performance = (alldata[['DeveloperID', 'DeveloperPerformance']]
               .groupby('DeveloperID').mean()
               .sort_values('DeveloperPerformance', ascending=False))
labels = [i for i in range(1,11)] # 1 to 10
performancelevel = pd.cut(performance['DeveloperPerformance'], 
                          bins=10, retbins=False, labels=labels)
performancelevel = (performancelevel
                    .reset_index()
                    .rename(columns={'DeveloperPerformance':'performancelevel'}))

def wrangler(X):
## data type conversion  
  X['EstimateOn'] = pd.to_datetime(X['EstimateOn'], format='%d-%b-%y')
  X['StartedOn'] = pd.to_datetime(X['StartedOn'], format='%d-%b-%y')
  X['CompletedOn'] = pd.to_datetime(X['CompletedOn'], format='%d-%b-%y')

## feature engineering
  X['hoursestimatelog'] = np.log1p(X['HoursEstimate'])
  X['daysactual'] = (X['CompletedOn']-X['StartedOn']).dt.days.abs()
  X['estimateonsameday'] = X['EstimateOn']==X['StartedOn']
  ## pandas.DataFrame.merge() behaves awkwardly in a function
  ## so I have to do it in this way
  tmp = (X[['ProjectCode']]
         .merge(right=breakdown, on='ProjectCode', how='left'))
  X['breakdown'] = tmp['ProjectBreakdownCode']
  tmp = (X[['DeveloperID']].merge(right=performancelevel,
                                  on='DeveloperID', how='left'))
  X['performancelevel'] = tmp['performancelevel'].astype(int)

## missing values
  X.replace([np.inf, -np.inf, pd.NaT], np.nan, inplace=True)
wrangler(alldata)
# alldata.to_csv("alldata.csv")

hoursactual_log = np.log1p(alldata['HoursActual'])
hoursestimate_log = np.log1p(alldata['HoursEstimate'])

In [0]:
print(alldata.columns.to_list())

['TaskNumber', 'Summary', 'Priority', 'RaisedByID', 'AssignedToID', 'AuthorisedByID', 'StatusCode', 'ProjectCode', 'ProjectBreakdownCode', 'Category', 'SubCategory', 'HoursEstimate', 'HoursActual', 'DeveloperID', 'DeveloperHoursActual', 'TaskPerformance', 'DeveloperPerformance', 'EstimateOn', 'StartedOn', 'CompletedOn', 'hoursestimatelog', 'daysactual', 'estimateonsameday', 'breakdown', 'performancelevel']


# Plotly

https://community.plot.ly/t/introducing-plotly-py-4-0-0/25639   
https://plot.ly/python/figure-factory-subplots/  
https://plot.ly/python/distplot/  

Plotly distplot subplot example  
https://nbviewer.jupyter.org/gist/empet/7eaf06b3cb5e488b129bb8df8fdb9b4b   
Formatting ticks  
https://plot.ly/python/v3/tick-formatting/  

In [0]:
# plotly distplot
hist_data = pd.DataFrame([hoursactual_log, hoursestimate_log])
path = 'hist_data.csv'
hist_data.to_csv(path)

hist_data = pd.read_csv(path, index_col=0).values
group_labels = ['Hours Actual (log)', 'Hours Estimated (log)']
fig = ff.create_distplot(hist_data, group_labels, 
                         bin_size=0.1, show_rug=False)

fig.update_layout(
    title=go.layout.Title(
        text="Software Development Effort Distribution",
        xref="paper", x=0),
    margin=dict(l=1,r=1,t=30,b=1),  
    legend=dict(x=.85, y=.95),
    xaxis=go.layout.XAxis(
        tickmode = 'array',
        tickvals = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        ticktext = ['Hours', 'e<br>(2.72 hours)', 'e^2<br>(7.39 hours)', 
                    'e^3<br>(20.09 hours)', 'e^4<br>(54.60 hours)', 'e^5<br>(148.41 hours)', 
                    'e^6<br>(403.43 hours)', 'e^7<br>(1096.63 hours)', 'e^8<br>(2980.96 hours)', 
                    'e^9<br>(8103.08 hours)']),
)

In [0]:
fig['data']

(Histogram({
     'autobinx': False,
     'histnorm': 'probability density',
     'legendgroup': 'Hours Actual (log)',
     'marker': {'color': 'rgb(31, 119, 180)'},
     'name': 'Hours Actual (log)',
     'opacity': 0.7,
     'x': array([1.01160091, 2.07944154, 0.53062825, ..., 2.1517622 , 0.47623418,
                 0.40546511]),
     'xaxis': 'x',
     'xbins': {'end': 7.820503744431879, 'size': 0.1, 'start': 0.009950330853168085},
     'yaxis': 'y'
 }), Histogram({
     'autobinx': False,
     'histnorm': 'probability density',
     'legendgroup': 'Hours Estimated (log)',
     'marker': {'color': 'rgb(255, 127, 14)'},
     'name': 'Hours Estimated (log)',
     'opacity': 0.7,
     'x': array([2.7080502 , 2.07944154, 0.53062825, ..., 1.5040774 , 0.91629073,
                 0.40546511]),
     'xaxis': 'x',
     'xbins': {'end': 6.814542897259956, 'size': 0.1, 'start': 0.009950330853168085},
     'yaxis': 'y'
 }), Scatter({
     'legendgroup': 'Hours Actual (log)',
     'marker': {'

# plotly table

https://plot.ly/python/table/  

In [0]:
url = "https://raw.githubusercontent.com/Nov05/DS-Unit-2-Sprint-4-Project/master/assets/hours_category.csv"
df = pd.read_csv(url)
print(df.columns.to_list())

['Unnamed: 0', 'Category', 'SubCategory', 'TaskNumber', 'HoursEstimateSum', 'HoursActualSum', 'HoursEstimateMax', 'HoursActualMax', 'HoursEstimateMin', 'HoursActualMin']


In [0]:
url = "https://raw.githubusercontent.com/Nov05/DS-Unit-2-Sprint-4-Project/master/assets/hours_category.csv"
path = "assets/hours_category.csv"
df = pd.read_csv(url)
df = df.round(2)

fig = go.Figure(data=[go.Table(
    header=dict(values=df.columns.to_list()[1:],
                align='left'),
    cells=dict(values=[df['Category'], 
                       df['SubCategory'], 
                       df['TaskNumber'], 
                       df['HoursEstimateSum'],
                       df['HoursActualSum'],
                       df['HoursEstimateMax'],
                       df['HoursActualMax'],
                       df['HoursEstimateMin'],
                       df['HoursActualMin'],
                      ],
               align='left')),
])
fig.update_layout(margin=dict(l=1,r=1,t=30,b=1),
                 )
# fig.show()

# Plotly Scatterplot

https://plot.ly/python/line-and-scatter/#scatter-and-line-plot-with-goscatter  

In [6]:
# read data
url = "https://raw.githubusercontent.com/Nov05/DS-Unit-2-Sprint-4-Project/master/assets/hours_predict.csv"
hours = pd.read_csv(url, index_col=0)
print(hours.shape)
hours.head()

(12299, 4)


,HoursActual,HoursEstimate,HoursPredict,HoursPredict2
0,1.011601,2.708050,2.268362,2.268362
1,2.079442,2.079442,1.165664,1.165664
2,0.530628,0.530628,1.860950,1.860950
3,0.530628,0.530628,1.208245,1.208245
4,1.504077,1.504077,1.696446,1.696446


In [10]:
hist_data = hours[['HoursActual', 'HoursEstimate']].T.values
group_labels = ['Hours Actual (log)', 'Hours Estimated (log)']
fig = ff.create_distplot(hist_data, group_labels, bin_size=0.1, show_rug=False)
fig.update_layout(
    title=go.layout.Title(text="Software Development Effort Distribution", xref="paper", x=0),
    margin=dict(l=1,r=1,t=30,b=1),  
    legend=dict(x=.65, y=.95),
    xaxis=go.layout.XAxis(
        tickmode = 'array',
        tickvals = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        ticktext = ['Hours', 'e<br>2.72 hours', 'e^2<br>7.39 hours', 
                    'e^3<br>20.09 hours', 'e^4<br>54.60 hours', 'e^5<br>148.41 hours', 
                    'e^6<br>403.43 hours', 'e^7<br>1096.63 hours', 'e^8<br>2980.96 hours', 
                    'e^9<br>8103.08 hours']),
)

In [0]:
def plotly_errors(hoursactual, hoursestimate, title):
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=hoursactual, 
                           y=hoursestimate,
                           mode='markers', 
                           hovertemplate="Actual: %{x:.2f}<br>Esitmated: %{y:.2f}",
                           name='effort hours (log)',
                           marker=dict(size=6, opacity=0.6,
                                       color=hoursestimate-hoursactual,
                                       colorscale='RdBu',
                                       line=dict(width=0.5, color='DarkSlateGrey'),
                                       showscale=True,
                                       reversescale=True,
                                       colorbar=dict(title='Error (log)',
                                                     thickness=5,
                                                     outlinewidth=0,),
                                      ),
                          )
               )
  fig.add_trace(go.Scatter(x=[0,8], y=[0,8],
                           mode='lines',
                           line=dict(color='DarkSlateGrey', width=2, dash='dash'),
                           name='perfect estimation'))
  fig.update_layout(title=title,
                    autosize=False, width=550, height=500,
                    margin=dict(l=1,r=1,t=30,b=1),
                    legend=dict(x=.05, y=.95),
                    xaxis=go.layout.XAxis(title=go.layout.xaxis.Title(text="Actual Hours (log)")),
                    yaxis=go.layout.YAxis(title=go.layout.yaxis.Title(text="Estimated Hours (log)")),
                   )
  fig.show()

plotly_errors(hours['HoursActual'], hours['HoursEstimate'], 
              "Manueal Estimation")

In [0]:
plotly_errors(hours['HoursActual'], hours['HoursPredict'], 
              "Model Prediction")

In [0]:
###############################################
# Load Pipeline
###############################################
from joblib import load
url = "https://github.com/Nov05/DS-Unit-2-Sprint-4-Project/blob/master/assets/pipeline.joblib"
path = "asset/pipeline.joblib"
pipeline = load(url) 

# total features: 5 ['Priority', 'Category', 'SubCategory', 
#                    'hoursestimatelog', 'breakdown']
# pred_trainval = pipeline.predict(X_trainval)

# Dash Options

In [0]:
opt_prior = alldata['Priority'].value_counts().index.to_list(); print(opt_prior)
opt_cate = alldata['Category'].value_counts().index.to_list(); print(opt_cate)
for c in opt_cate:
    print(c, alldata[alldata['Category']==c]['SubCategory'].value_counts().index.to_list())
print(min(alldata['HoursActual']), max(alldata['HoursActual']))
print(min(alldata['breakdown']), max(alldata['breakdown']))
opt_perf = alldata['performancelevel'].value_counts().index.to_list(); opt_perf

In [0]:
# Dash Input options
# convert list to dictionary
opt_cate = ['Development', 'Management', 'Operational']
opt_sub_dev = ['Enhancement', 'Bug', 'Support', 'Release', 'Training', 
               'Testing', 'Business Specification', 'Conversion', 
               'Technical Specification', 'Research', 'Documentation', 'Third Party']
opt_sub_mngt = ['Marketing', 'Management Meeting', 'General Documentation', 'Progress Meeting', 
                'Staff Management', 'Office Management', 'Project Management', 'Client Support', 
                'Board Meeting', 'Staff Recruitment']
opt_sub_oprt = ['In House Support', 'Client Support', 'Consultancy', 'Documentation']

def list_to_opt(l):
    opt = []
    for i in l:
        opt.append({'label':i, 'value':i})
    return opt

print(list_to_opt(opt_cate))